In [62]:
import pymongo
import pandas as pd

### 下載檔案

In [2]:
import urllib.request
res = "https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-D0047-093?Authorization=CWB-73638260-F88A-444D-837B-498A220E9B8E&downloadType=WEB&format=ZIP"
urllib.request.urlretrieve(res, './data/F-D0047-093.zip')

('./data/F-D0047-093.zip', <http.client.HTTPMessage at 0x7fc0a0bb9410>)

### 解壓縮

In [3]:
import zipfile
f = zipfile.ZipFile('./data/F-D0047-093.zip')
f.extractall('./data')

### 抓所有名為xxxx_Week24_CH的檔案

In [4]:
import os
data = []
path = './data/'
file = os.listdir(path)
file.remove('TAIWAN_Week24_CH.xml')
for k in file:
    if '_Week24_CH' in k:
        data.append(k)

### 迴圈將資料抓進Dict

In [94]:
from bs4 import BeautifulSoup
import xmltodict
city = []
town = []
date = []
temp = []
for l in data:
    fh = open(path+l, "r")
    xml = fh.read()
    d = dict(xmltodict.parse(xml))
    soup = BeautifulSoup(xml,'xml')
    for j in range(0,len(soup.find_all('locationName'))):        
        all_avg_temp = []
        for i in range(0,6):
            city.append(d['cwbopendata']['dataset']['locations']['locationsName'])
            town.append(d['cwbopendata']['dataset']['locations']['location'][j]['locationName'])
            date.append(d['cwbopendata']['dataset']['locations']['location'][j]['weatherElement'][k]['time'][i]['startTime'][:10])
            temp.append(d['cwbopendata']['dataset']['locations']['location'][j]['weatherElement'][k]['time'][i]['elementValue']['value'])
result = {
    'city':city,
    'town':town,
    'date':date,
    'temp':temp
}

In [95]:
df = pd.DataFrame(result)
df

,city,town,date,temp
0,桃園市,桃園區,2022-08-16,34
1,桃園市,桃園區,2022-08-17,32
2,桃園市,桃園區,2022-08-18,32
3,桃園市,桃園區,2022-08-19,32
4,桃園市,桃園區,2022-08-20,31
...,...,...,...,...
2203,南投縣,仁愛鄉,2022-08-17,20
2204,南投縣,仁愛鄉,2022-08-18,20
2205,南投縣,仁愛鄉,2022-08-19,20
2206,南投縣,仁愛鄉,2022-08-20,20


### 開檔/讀檔

In [96]:
## 建立 MongoDB 連線

client = pymongo.MongoClient('localhost', 27017)

### 將 DataFrame 插入 Mongo

In [97]:
database = client["mongo-sample"] # SQL: Database Name 
collection = database["Week24_temp"] # SQL: Table Name

records = df.to_dict('records') 
collection.insert_many(records)

### 將 Mongo 存入 Pandas

In [98]:

database = client["mongo-sample"] # SQL: Database Name 
collection = database["Week24_temp"] # SQL: Table Name

query = {'date': '2022-08-19'}
records = pd.DataFrame(list(collection.find(query)))
records

,_id,city,town,date,temp
0,62fb330c3e3ed0612fbc9762,桃園市,桃園區,2022-08-19,32
1,62fb330c3e3ed0612fbc9768,桃園市,中壢區,2022-08-19,31
2,62fb330c3e3ed0612fbc976e,桃園市,大溪區,2022-08-19,30
3,62fb330c3e3ed0612fbc9774,桃園市,楊梅區,2022-08-19,29
4,62fb330c3e3ed0612fbc977a,桃園市,蘆竹區,2022-08-19,32
...,...,...,...,...,...
363,62fb330c3e3ed0612fbc9fe4,南投縣,魚池鄉,2022-08-19,24
364,62fb330c3e3ed0612fbc9fea,南投縣,國姓鄉,2022-08-19,27
365,62fb330c3e3ed0612fbc9ff0,南投縣,水里鄉,2022-08-19,27
366,62fb330c3e3ed0612fbc9ff6,南投縣,信義鄉,2022-08-19,25
